In [17]:
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments()
experiments


[<Experiment: artifact_location='file:///home/guess/Desktop/scanflow/scanflow/special/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [24]:
type(client.get_experiment_by_name('Defaulst'))

NoneType

In [19]:
def search_experiment(name):
    for e in experiments:
        if e.name == name:
            return e
        
search_experiment('Default')

<Experiment: artifact_location='file:///home/guess/Desktop/scanflow/scanflow/special/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>

In [123]:
runs_info = client.list_run_infos('0', order_by=["attribute.start_time DESC"])
runs_info


[<RunInfo: artifact_uri='file:///home/guess/Desktop/scanflow/scanflow/special/mlruns/0/688316478a4240edac605583121c6f05/artifacts', end_time=1615848505110, experiment_id='0', lifecycle_stage='active', run_id='688316478a4240edac605583121c6f05', run_uuid='688316478a4240edac605583121c6f05', start_time=1615848505101, status='FINISHED', user_id='guess'>]

In [127]:
last_run_id = runs_info[0].run_id
last_run_id

'688316478a4240edac605583121c6f05'

In [128]:
client.list_artifacts(last_run_id)

[<FileInfo: file_size=None, is_dir=True, path='Input'>]

In [64]:
# client.get_run(run_id)

In [130]:
last_artifacts = client.list_artifacts(last_run_id, path='Input')
last_artifacts

[<FileInfo: file_size=56, is_dir=False, path='Input/input_df.csv'>]

In [93]:
%%time

import json
import os

def get_feedback(last_artifacts):
    for last_artifact in last_artifacts:
        if 'feedback' in last_artifact.path:
            return last_artifact.path
    
feedback_artif_path = get_feedback(last_artifacts)
print(feedback_artif_path)
client.download_artifacts(run_id, 
                          feedback_artif_path,
                          '/tmp/')

feedback_local_path = os.path.join('/tmp', feedback_artif_path)
with open(feedback_local_path) as fread:
    data = json.load(fread)

print(data)

Anomaly/feedback_anomaly.json
{'n_anomalies': 3, 'percentage_anomalies': 0.6}
CPU times: user 5.17 ms, sys: 0 ns, total: 5.17 ms
Wall time: 4.56 ms


In [117]:
import numpy as np

np.random.choice([0, 1], size=(10,), p=[2./3, 1./3])

array([1, 1, 0, 1, 0, 1, 0, 1, 0, 0])

In [133]:
import pandas as pd

n_input = 10
d_input = {'input': [f"img{n}.jpg" for n in range(n_input)]}
input_df = pd.DataFrame(d_input)
input_df

,input
0,img0.jpg
1,img1.jpg
2,img2.jpg
3,img3.jpg
4,img4.jpg
5,img5.jpg
6,img6.jpg
7,img7.jpg
8,img8.jpg
9,img9.jpg


In [134]:
input_df.to_csv('input.csv', index=False)
with mlflow.start_run(experiment_id='0',
                      run_name='input') as mlrun:
    mlflow.log_artifact('input.csv', 'Input')


In [100]:
client.download_artifacts(run_id, 'Anomaly/feedback_anomaly.json', 
                          '/home/guess/Downloads')

'/home/guess/Downloads/Anomaly/feedback_anomaly.json'

In [152]:
import aiohttp
import requests

url = 'http://localhost:8011/checker/anomaly'
content = {"run_id":"95e9076278b8430292f734dd4d67b340",
          "input": "Input/input.csv"}

async with aiohttp.ClientSession() as session:
    async with session.post(url, json=content) as response:
        result = await response.json()



result

{'feedback': {'n_anomalies': 1, 'percentage_anomalies': 0.1}}

In [1]:
import mlflow
mlflow.__version__

'1.14.1'

In [8]:
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor

mlflow.set_tracking_uri("sqlite:///mydb.sqlite")
params = {"n_estimators": 3, "random_state": 42}
name = "RandomForestRegression"
rfr = RandomForestRegressor(**params).fit([[0, 1]], [1])
# Log MLflow entities
with mlflow.start_run() as run:
    mlflow.log_params(params)
    mlflow.sklearn.log_model(rfr, artifact_path="sklearn-model")

# Register model name in the model registry
client = MlflowClient()
# client.create_registered_model(name)

# Create a new version of the rfr model under the registered model name
desc = "A new version of the model"
model_uri = "runs:/{}/sklearn-model".format(run.info.run_id)
mv = client.create_model_version(name, model_uri, run.info.run_id, description=desc)
print("Name: {}".format(mv.name))
print("Version: {}".format(mv.version))
print("Description: {}".format(mv.description))
print("Status: {}".format(mv.status))
print("Stage: {}".format(mv.current_stage))

2021/03/18 16:49:54 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestRegression, version 2


Name: RandomForestRegression
Version: 2
Description: A new version of the model
Status: READY
Stage: None


In [9]:
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

iris = datasets.load_iris()
iris_train = pd.DataFrame(iris.data, columns=iris.feature_names)
clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(iris_train, iris.target)
signature = infer_signature(iris_train, clf.predict(iris_train))
mlflow.sklearn.log_model(clf, artifact_path="iris_rf", signature=signature)

In [10]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import SGD

import mlflow
import mlflow.keras
from mlflow.models.signature import infer_signature

(train_X, train_Y), (test_X, test_Y) = mnist.load_data()
trainX = train_X.reshape((train_X.shape[0], 28, 28, 1))
testX = test_X.reshape((test_X.shape[0], 28, 28, 1))
trainY = to_categorical(train_Y)
testY = to_categorical(test_Y)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=1, batch_size=32, validation_data=(testX, testY))

signature = infer_signature(testX, model.predict(testX))
mlflow.keras.log_model(model, "mnist_cnn", signature=signature)

1875/1875 [==============================] - 18s 10ms/step - loss: 363.1356 - accuracy: 0.1087 - val_loss: 2.3015 - val_accuracy: 0.1135
INFO:tensorflow:Assets written to: /tmp/tmp9tok352c/model/data/model/assets
